# ExampleData의 가속도 센서 모두에 대해 <span style='background-color: #ffdce0'>수동으로 STFT 적용하기</span>

## ! 직접 Window를 나누어 FFT 적용 후 합치기

- 각 센서에 대해 100개씩 30개의 데이터로 분할 (각 센서 따로): Shape = (30, 100)
- 분할된 데이터에 FFT 적용 (각 센서 따로) -> (30, 50)
- 모든 데이터 합치기 (transpose, np.array().reshape(), concatenate 함수 사용) -> (50, 30, 1) + (50, 30,1) = (50, 30, 2)

## Quiz Guide

In [ ]:
# 구글 드라이브 접근 코드
from google.colab import drive
drive.mount('/content/drive')
My_path = '/content/drive/MyDrive/AI_Code/'

In [ ]:
import pandas as pd
import numpy  as np
import math
from scipy import signal
import matplotlib.pyplot as plt

In [ ]:
ExampleData = pd.read_csv(My_path + 'ExampleData.csv', sep=',', header=None)
ExampleData

In [ ]:
# 데이터 분할 (30, 100)

SplitData_1 = np.zeros([30, 100]) # 가속도 센서 1
SplitData_2 = np.zeros([30, 100]) # 가속도 센서 2

for i in range(30):
    SplitData_1[i, :] = ExampleData.iloc[i*100:(i+1)*100,1]
    SplitData_2[i, :] = ExampleData.iloc[i*100:(i+1)*100,2]

SplitData_1.shape, SplitData_2.shape

In [ ]:
# 분할된 데이터별 FFT 적용

Fs = 20000

Sxx_1 = np.zeros([30, 50]) # SplitData_1 각 FFT 결과
Sxx_2 = np.zeros([30, 50]) # SplitData_2 각 FFT 결과

# SplitData_1에 대한 FFT 수행
for i in range(SplitData_1.shape[0]):
    # 분할된 데이터 가져오기
    x      = SplitData_1[i, :]

    # FFT 적용
    NFFT   =  len(x)                         # length of signal
    k      =  np.arange(NFFT)
    f0     =  k * Fs / NFFT                  # Double sides Frequency range
    f0     =  f0[range(math.trunc(NFFT/2))]  # Single side  Frequency range

    Y      =  np.fft.fft(x)/NFFT             # FFT computing and normalization
    Y      =  Y[range(math.trunc(NFFT/2))]   # Single side  Frequency range
    amp_Hz =  2 * abs(Y)
    phase_ang = np.angle(Y) * 180/np.pi

    # FFT 결과 저장
    Sxx_1[i, :] = amp_Hz

# SplitData_2에 대한 FFT 수행
for i in range(SplitData_2.shape[0]):
    # 분할된 데이터 가져오기
    x      = SplitData_2[i, :]

    # FFT 적용
    NFFT   =  len(x)                         # length of signal
    k      =  np.arange(NFFT)
    f0     =  k * Fs / NFFT                  # Double sides Frequency range
    f0     =  f0[range(math.trunc(NFFT/2))]  # Single side  Frequency range

    Y      =  np.fft.fft(x)/NFFT             # FFT computing and normalization
    Y      =  Y[range(math.trunc(NFFT/2))]   # Single side  Frequency range
    amp_Hz =  2 * abs(Y)
    phase_ang = np.angle(Y) * 180/np.pi

    # FFT 결과 저장
    Sxx_2[i, :] = amp_Hz


# Transpose 필수! (아래 코드 두줄 수정 X)
Sxx_1 = Sxx_1.T
Sxx_2 = Sxx_2.T

# Reshape 함수 적용
Sxx_1 = Sxx_1.reshape([Sxx_1.shape[0], Sxx_1.shape[1], 1])
Sxx_2 = Sxx_2.reshape([Sxx_2.shape[0], Sxx_2.shape[1], 1])

# Total_Sxx로 합치기 (50, 30, 2)
Total_Sxx = np.concatenate([Sxx_1, Sxx_2], axis = -1)
Total_Sxx.shape

In [ ]:
Total_Sxx

In [ ]:
plt.figure(figsize=(9,6))
plt.pcolormesh(Total_Sxx[:,:,0], cmap = 'hot', alpha = 0.7)

plt.xlabel('Time(s)', fontsize=12)
plt.ylabel('Frequency(Hz)', fontsize=12)
plt.colorbar()
plt.grid(alpha = 0.4)

plt.show()

In [ ]:
x = ExampleData.iloc[:,1]

# f,t,Sxx = signal.spectrogram(x, Fs, nperseg = 100, noverlap=0)
f,t,Sxx = signal.spectrogram(x, Fs, nperseg = 100, window='boxcar', scaling='spectrum', mode='magnitude', noverlap=0)
f.shape, t.shape, Sxx.shape

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(1, 2, 1)

plt.pcolormesh(Sxx, vmin = 0, vmax = 0.2, cmap = 'hot', alpha = 0.7)
# plt.pcolormesh(Sxx[:-1]*2, vmin = 0, vmax = 0.2, cmap = 'hot', alpha = 0.7)

plt.title('Spectrogram (FFT with window)')
plt.xlabel('Time(s)', fontsize=12)
plt.ylabel('Frequency(Hz)', fontsize=12)
plt.grid(alpha = 0.4)

plt.subplot(1, 2, 2)
plt.pcolormesh(Total_Sxx[:,:,0], vmin = 0, vmax = 0.2, cmap = 'hot', alpha = 0.7)

plt.title('Spectrogram (STFT)')
plt.xlabel('Time(s)', fontsize=12)
plt.ylabel('Frequency(Hz)', fontsize=12)
plt.grid(alpha = 0.4)


plt.show()